In [1]:
import pandas as pd

data = pd.read_csv('../preprocess/202111171547_all.csv')
df = pd.DataFrame(data)
df = df.dropna()
data = df.iloc[:, :-1]
target = df['target']

In [2]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, stratify=target, random_state=0)
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2, stratify=train_target, random_state=0)

In [4]:
from tensorflow import keras
import tensorflow.keras.backend as K

K.clear_session()
model = keras.Sequential()
model.add(keras.layers.GRU(32, activation="tanh", input_shape=(train_input.shape[1],1), return_sequences=True))
model.add(keras.layers.GRU(16, activation="tanh", dropout=0.3))
model.add(keras.layers.Dense(1, activation="relu"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 160, 32)           3360      
                                                                 
 gru_1 (GRU)                 (None, 16)                2400      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 5,777
Trainable params: 5,777
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('11241550-best-gru-model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights=True)

history = model.fit(train_input, train_target, 
                    epochs=100, batch_size = 50,
                    validation_data = (val_input, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
64/64 [==============================] - 8s 78ms/step - loss: 0.9473 - accuracy: 0.6026 - val_loss: 0.5961 - val_accuracy: 0.8616
Epoch 2/100
64/64 [==============================] - 4s 69ms/step - loss: 0.6626 - accuracy: 0.7725 - val_loss: 0.4793 - val_accuracy: 0.8415
Epoch 3/100
64/64 [==============================] - 4s 70ms/step - loss: 0.5776 - accuracy: 0.8005 - val_loss: 0.5842 - val_accuracy: 0.8616
Epoch 4/100
64/64 [==============================] - 4s 68ms/step - loss: 0.6031 - accuracy: 0.8084 - val_loss: 0.6331 - val_accuracy: 0.8679
Epoch 5/100
64/64 [==============================] - 4s 67ms/step - loss: 0.6023 - accuracy: 0.8317 - val_loss: 0.6721 - val_accuracy: 0.7937
Epoch 6/100
64/64 [==============================] - 4s 68ms/step - loss: 0.6347 - accuracy: 0.8424 - val_loss: 0.5893 - val_accuracy: 0.8642
Epoch 7/100
64/64 [==============================] - 5s 71ms/step - loss: 0.6064 - accuracy: 0.8367 - val_loss: 0.6131 - val_accuracy: 0.8616
Epoch 

In [19]:
model.evaluate(val_input, val_target)

25/25 [==============================] - 0s 14ms/step - loss: 0.3092 - accuracy: 0.8717


[0.30923259258270264, 0.8716981410980225]

In [20]:
model.evaluate(test_input, test_target)

32/32 [==============================] - 0s 13ms/step - loss: 0.3326 - accuracy: 0.8934


[0.33256348967552185, 0.8933601379394531]